In [1]:
import boto3
import io
import json
import redis

In [10]:
bucket_name = "brainshare-primary-6944fc2"
object_key = "entries.idx"
s3 = boto3.client("s3")

In [11]:
buffer = io.BytesIO()

In [12]:
s3.download_fileobj(bucket_name, object_key, buffer)

In [13]:
buffer.seek(0)

0

In [14]:
file_content = buffer.read()

In [15]:
print(f"File loaded into memory. Content length: {len(file_content)} bytes")

File loaded into memory. Content length: 50760965 bytes


In [16]:
ind = file_content.decode("utf-8")

In [17]:
import pandas as pd

In [18]:
file_like_object = io.StringIO(ind)

In [19]:
header = ind.split("\n")[0].split(", ")

In [20]:
df = pd.read_csv(file_like_object, delimiter="\t", skiprows=2, names=header)

In [21]:
len(df)

224612

In [22]:
pdb_ids = df.iloc[:, 0]

In [4]:
r = redis.Redis.from_url("redis://localhost:6379")

In [5]:
try:
    r.ping()
    print("Connected to Redis!")
except redis.ConnectionError as e:
    print(f"Failed to connect: {e}")

Connected to Redis!


In [6]:
redis_set_key = "br-resource-pdb-ids"

In [23]:
r.sadd(redis_set_key, *pdb_ids)

224612

In [24]:
r.scard(redis_set_key)

224612

In [ ]:
redis_info_key = "br-resource-info-pdb-ids"
r.set(
    redis_info_key,
    json.dumps(
        {
            "num_entries": len(pdb_ids),
            "description": "PDB IDs for all entries in the BrainShare resource",
            "link": "https://www.rcsb.org/",
            "link_prefix": "https://www.rcsb.org/structure/",
        }
    ),
)